In [4]:
import numpy as np
import math
import matplotlib.pyplot as plt

In [5]:
cuda = 7
folder_name = 'sev'
device = torch.device('cuda:%s'%cuda)
stage_scale_list = []
def cal_integrate_gradients():
    print(mice_id)
    attribution_array1, attribution_array2  = [], []
    train_input_list, train_target_list, test_input_list, test_target_list = [], [], [], []
    test_input = pd.read_csv('./loc_data_allstage/%s/%s/stage%s/test_input.csv'%(folder_name, mice_id, stage), header=None)
    test_input_list.append(np.array(test_input))
    test_target = pd.read_csv('./loc_data_allstage/%s/%s/stage%s/test_target.csv'%(folder_name, mice_id, stage), header=None)
    test_target_list.append(np.array(test_target))
    test_input_list = np.concatenate(np.array(test_input_list),axis=0)
    test_target_list = np.concatenate(np.array(test_target_list),axis=0)
    input_data = torch.tensor(test_input_list, dtype=torch.float32, device=device) # [1,1000,64]
    target_data = torch.tensor(test_target_list, dtype=torch.float32, device=device) # [1,1000,64]
    input = input_data[:800]
    target = target_data[:800]
    hidden_units = 100
    scale = 1
    batch_size = 512
    MAE = torch.nn.L1Loss(reduction='none')
    MSE = torch.nn.MSELoss(reduction='none')
    #Define the net
    scale_std = []
    scale_list = [32,16,8,4,2,1]
    value_index = range(32)
    net = Parellel_Renorm_Dynamic(sym_size = input_data.shape[1], latent_size = scale, effect_size = input_data.shape[1],cut_size=2, hidden_units = 100, normalized_state = True, device=device)
    net = net.to(device)
    net.load_state_dict(torch.load(f'./loc_model_stage2/{folder_name}/{mice_id}/stage{stage}_scale%s.pkl'%weight_id))        
    feature_number = scale_list[weight_id]
    

    def encoding_function(input_data):
        return net.encoding2(input_data, weight_id)[-1]
        
        ig = IntegratedGradients(encoding_function)
        group_number = pd.read_csv('./loc_data_allstage/%s/%s/%s/group_number.csv'%(folder_name, mice_id, 'stage1'), header=None)
        brain_id = pd.read_csv('./loc_data_allstage/%s/%s/%s/brain_id.csv'%(folder_name, mice_id, 'stage1'), header=None)
        loc_data = pd.read_csv('./loc_data_allstage/%s/%s/%s/mean_loc.csv'%(folder_name, mice_id, 'stage1'), header=None)
        cluster_idx = pd.read_csv('./loc_data_allstage/%s/%s/%s/cluster_idx.csv'%(folder_name, mice_id, 'stage1'), header=None)
        loc_axis = pd.read_csv('./loc_data_allstage/%s/%s/%s/loc_axis.csv'%(folder_name, mice_id, 'stage1'), header=None)
        
        loc_data = np.array(loc_data)
        cluster_idx = np.array(cluster_idx)
        loc_axis = np.array(loc_axis)
        group_number = np.array(group_number)[:,0]
        brain_id = np.array(brain_id)[:,0]
        group_cum = np.cumsum(group_number)
        group_cum = np.concatenate((np.array([0]), group_cum))
        attri_neuro = np.zeros(cluster_idx.shape[0])
        
        brain_area_number = list(dict(count_and_sort_elements(brain_id)).values())
        temp_std = []
        for target in range(feature_number):
            attributions, approximation_error = ig.attribute(input,target=target,method='gausslegendre',return_convergence_delta=True)
            attributions = (attributions.abs().cpu().numpy()).mean(0)
        attribution_array1.append(attributions)
        for temp_id in range(len(group_cum)-1):
            # attri_neuro[group_cum[temp_id]:group_cum[temp_id+1]] = attributions[temp_id] / group_number[temp_id]
            attri_neuro[group_cum[temp_id]:group_cum[temp_id+1]] = attributions[temp_id]
        attribution_array2.append(attri_neuro)
attribution_array1_all.append(attribution_array1)
attribution_array2_all.append(attribution_array2)
    
# attribution_array1_all = np.array(attribution_array1_all)
# attribution_array2_all = np.array(attribution_array2_all)